# Cross-Workspace Data Access (Analytics Workspace)

**Workspace**: Analytics/Data Science (Secondary)  
**Purpose**: Access and analyze data created in the Primary workspace  
**Catalog**: `shared_data` (shared across workspaces)

This notebook demonstrates:
- Accessing Delta tables from another workspace via Unity Catalog
- Cross-workspace data sharing without data duplication
- Centralized governance and permissions
- Analytics and ML on shared data

## 1. Verify Unity Catalog Access

In [ ]:
# Check workspace information
workspace_url = spark.conf.get("spark.databricks.workspaceUrl")
print(f"Current Workspace: {workspace_url}")
print(f"This is the ANALYTICS workspace\n")

# Verify Unity Catalog is enabled
uc_enabled = spark.conf.get("spark.databricks.unity_catalog.enabled", "false")
print(f"Unity Catalog Enabled: {uc_enabled}")

In [ ]:
# List available catalogs
print("Available Catalogs (accessible from this workspace):")
display(spark.sql("SHOW CATALOGS"))

In [ ]:
# Switch to shared catalog
spark.sql("USE CATALOG shared_data")
spark.sql("USE SCHEMA samples")

print("✅ Connected to shared_data.samples")
print("   This catalog was created in the PRIMARY workspace")
print("   But is accessible here via Unity Catalog!")

## 2. Explore Available Tables

In [ ]:
# List tables in the shared schema
print("Tables available in shared_data.samples:")
display(spark.sql("SHOW TABLES IN shared_data.samples"))

In [ ]:
# Get detailed information about tables
tables = ["customers", "products", "transactions"]

for table in tables:
    full_table_name = f"shared_data.samples.{table}"
    print(f"\n{'='*60}")
    print(f"Table: {full_table_name}")
    print(f"{'='*60}")
    
    # Get table details
    df = spark.table(full_table_name)
    print(f"Record Count: {df.count():,}")
    print(f"\nSchema:")
    df.printSchema()
    
    # Show sample data
    print(f"\nSample Data (first 3 rows):")
    display(df.limit(3))

## 3. Query Shared Data - Customer Analysis

In [ ]:
# Customer analysis
customers_df = spark.table("shared_data.samples.customers")

print("Customer Distribution by Country:")
display(
    customers_df
    .groupBy("country")
    .agg(
        count("*").alias("customer_count"),
        sum("lifetime_value").alias("total_lifetime_value"),
        avg("lifetime_value").alias("avg_lifetime_value")
    )
    .orderBy(desc("total_lifetime_value"))
)

In [ ]:
# Top customers by lifetime value
print("Top 5 Customers by Lifetime Value:")
display(
    customers_df
    .select("customer_name", "country", "total_purchases", "lifetime_value")
    .orderBy(desc("lifetime_value"))
    .limit(5)
)

## 4. Cross-Table Analysis - Transaction Insights

In [ ]:
# Join customers, transactions, and products
from pyspark.sql.functions import col, sum as _sum, count, avg, round as _round

customers = spark.table("shared_data.samples.customers")
transactions = spark.table("shared_data.samples.transactions")
products = spark.table("shared_data.samples.products")

# Create comprehensive transaction view
transaction_analysis = (
    transactions
    .join(customers, "customer_id")
    .join(products, "product_id")
    .select(
        col("transaction_id"),
        col("customer_name"),
        col("country"),
        col("product_name"),
        col("category"),
        col("quantity"),
        col("total_amount"),
        col("transaction_date"),
        col("payment_method")
    )
)

print("Complete Transaction Analysis:")
display(transaction_analysis.orderBy(desc("transaction_date")))

In [ ]:
# Revenue by product category
print("Revenue Analysis by Product Category:")
display(
    transaction_analysis
    .groupBy("category")
    .agg(
        count("*").alias("transaction_count"),
        _sum("total_amount").alias("total_revenue"),
        _sum("quantity").alias("total_units_sold"),
        _round(avg("total_amount"), 2).alias("avg_transaction_value")
    )
    .orderBy(desc("total_revenue"))
)

In [ ]:
# Payment method preferences
print("Payment Method Distribution:")
display(
    transaction_analysis
    .groupBy("payment_method")
    .agg(
        count("*").alias("transaction_count"),
        _sum("total_amount").alias("total_amount"),
        _round(avg("total_amount"), 2).alias("avg_amount")
    )
    .orderBy(desc("transaction_count"))
)

## 5. Use the Pre-created View

In [ ]:
# Query the view created in the Primary workspace
print("Querying shared_data.samples.customer_transactions view:")
print("(This view was created in the PRIMARY workspace)\n")

customer_trans_view = spark.table("shared_data.samples.customer_transactions")
display(customer_trans_view.orderBy(desc("transaction_date")).limit(10))

## 6. Create Analytics-Specific Objects

In [ ]:
# Create a materialized aggregate table for analytics
spark.sql("""
CREATE OR REPLACE TABLE shared_data.samples.customer_summary AS
SELECT 
    c.customer_id,
    c.customer_name,
    c.country,
    c.signup_date,
    COUNT(t.transaction_id) as transaction_count,
    SUM(t.total_amount) as total_spent,
    AVG(t.total_amount) as avg_transaction_value,
    MAX(t.transaction_date) as last_purchase_date,
    DATEDIFF(CURRENT_DATE(), MAX(t.transaction_date)) as days_since_last_purchase
FROM shared_data.samples.customers c
LEFT JOIN shared_data.samples.transactions t ON c.customer_id = t.customer_id
GROUP BY c.customer_id, c.customer_name, c.country, c.signup_date
""")

print("✅ Created analytics table: shared_data.samples.customer_summary")

In [ ]:
# View the customer summary
display(
    spark.table("shared_data.samples.customer_summary")
    .orderBy(desc("total_spent"))
)

## 7. Machine Learning Feature Preparation

In [ ]:
# Prepare features for ML model (customer segmentation)
from pyspark.sql.functions import when, datediff, current_date, months_between

ml_features = (
    spark.table("shared_data.samples.customer_summary")
    .withColumn(
        "customer_segment",
        when(col("total_spent") >= 20000, "Premium")
        .when(col("total_spent") >= 10000, "Gold")
        .when(col("total_spent") >= 5000, "Silver")
        .otherwise("Bronze")
    )
    .withColumn(
        "recency_score",
        when(col("days_since_last_purchase") <= 30, 5)
        .when(col("days_since_last_purchase") <= 60, 4)
        .when(col("days_since_last_purchase") <= 90, 3)
        .when(col("days_since_last_purchase") <= 180, 2)
        .otherwise(1)
    )
    .withColumn(
        "frequency_score",
        when(col("transaction_count") >= 10, 5)
        .when(col("transaction_count") >= 7, 4)
        .when(col("transaction_count") >= 5, 3)
        .when(col("transaction_count") >= 3, 2)
        .otherwise(1)
    )
    .withColumn(
        "monetary_score",
        when(col("total_spent") >= 20000, 5)
        .when(col("total_spent") >= 10000, 4)
        .when(col("total_spent") >= 5000, 3)
        .when(col("total_spent") >= 2000, 2)
        .otherwise(1)
    )
)

print("Customer Segmentation with RFM Scores:")
display(
    ml_features
    .select(
        "customer_name",
        "country",
        "customer_segment",
        "recency_score",
        "frequency_score",
        "monetary_score",
        "total_spent",
        "transaction_count"
    )
    .orderBy(desc("monetary_score"), desc("frequency_score"), desc("recency_score"))
)

## 8. Demonstrate Unity Catalog Permissions

In [ ]:
# Show current permissions (if you have admin access)
try:
    print("Grants on shared_data catalog:")
    display(spark.sql("SHOW GRANTS ON CATALOG shared_data"))
except Exception as e:
    print(f"Note: Unable to show grants (requires admin permissions)\n{e}")

In [ ]:
# Check table history (Delta Lake feature)
print("Transaction History (Delta Lake Time Travel):")
display(spark.sql("DESCRIBE HISTORY shared_data.samples.transactions"))

## 9. Data Quality Checks

In [ ]:
# Run data quality checks on shared data
def run_data_quality_checks(table_name):
    print(f"\n{'='*60}")
    print(f"Data Quality Report: {table_name}")
    print(f"{'='*60}\n")
    
    df = spark.table(table_name)
    
    # Basic stats
    total_rows = df.count()
    print(f"✓ Total Rows: {total_rows:,}")
    
    # Check for nulls in each column
    print(f"\n✓ Null Value Check:")
    for col_name in df.columns:
        null_count = df.filter(col(col_name).isNull()).count()
        if null_count > 0:
            print(f"   ⚠️  {col_name}: {null_count} nulls ({null_count/total_rows*100:.2f}%)")
        else:
            print(f"   ✓ {col_name}: No nulls")
    
    # Duplicate check (if ID column exists)
    id_cols = [c for c in df.columns if 'id' in c.lower()]
    if id_cols:
        id_col = id_cols[0]
        unique_count = df.select(id_col).distinct().count()
        print(f"\n✓ Uniqueness Check ({id_col}):")
        if unique_count == total_rows:
            print(f"   ✓ All {id_col} values are unique")
        else:
            print(f"   ⚠️  Found {total_rows - unique_count} duplicate {id_col} values")

# Run checks on all tables
for table in ["customers", "products", "transactions"]:
    run_data_quality_checks(f"shared_data.samples.{table}")

## 10. Summary - Unity Catalog Benefits Demonstrated

### ✅ **Cross-Workspace Data Access**
- Accessed tables created in **PRIMARY workspace** from **ANALYTICS workspace**
- No data duplication or complex data pipelines needed
- Single source of truth with centralized governance

### ✅ **Centralized Metadata**
- Three-level namespace: `catalog.schema.table`
- Consistent access across all workspaces
- Unified data discovery and lineage

### ✅ **Security & Governance**
- Fine-grained permissions at catalog, schema, and table level
- Row and column-level security (can be configured)
- Audit logging of all data access

### ✅ **Delta Lake Features**
- ACID transactions
- Time travel and versioning
- Schema evolution
- Optimization capabilities

### 📊 **What We Created**
- Queried 3 shared tables: customers, products, transactions
- Used pre-created view from Primary workspace
- Created new analytics table: customer_summary
- Prepared ML features with RFM segmentation
- Performed data quality checks

### 🚀 **Next Steps**
- Set up row-level security
- Implement column masking for PII data
- Create automated data quality pipelines
- Build ML models using shared features
- Set up data lineage visualization